# Minneapolis, MN Simulations
8760 rows per simulation<br>
196 files (removed cell-in-cell)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 0.83
elec = 0.1314

---

## Combine CSV Files in Directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/msp/") if f.endswith(".csv")]

# files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/msp/" + file)
    all_data = pd.concat([all_data, df])
    
#all_data

# 1716960  rows, 73 columns

In [5]:
# all_data.dtypes

In [6]:
# Get wanted columns
all_data1 = all_data[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "]]

#all_data1

In [7]:
all_data1 = all_data1.dropna().reset_index(drop=True)

#all_data1

#1716960  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [8]:
# Rename Columns
all_data2 = all_data1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data2

#1716960  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [9]:
# Copy dataframe to modify and leave original df intact
get_annual = all_data2.copy(deep=True)

#get_annual

In [10]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual

In [11]:
# Drop columns
get_annual.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual

---

## Get Annual Cost Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [12]:
# Get annual costs
get_annual_costs = all_data2.copy(deep=True)

In [13]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanCost($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual_costs["HeatingCost($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual_costs["CoolingCost($)"] = (get_annual_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [14]:
# Drop columns
get_annual_costs.drop(columns = ["Illuminance(lux)","FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Add baseline data <br>
Export combine file

In [15]:
# Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs)

#allAnnualData

In [16]:
# Add energy and cost totals
totalEnergy = allAnnualData["FanEnergy(kBtu)"] + allAnnualData["HeatingEnergy(kBtu)"] + allAnnualData["CoolingEnergy(kBtu)"]
allAnnualData["AnnualEnergy(kBtu)"] = totalEnergy

totalCost = allAnnualData["FanCost($)"] + allAnnualData["HeatingCost($)"] + allAnnualData["CoolingCost($)"]
allAnnualData["AnnualCost($)"] = totalCost

#allAnnualData


### Establish Annual Energy and Cost Baseline Data


In [17]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_annual = allAnnualData[allAnnualData["Scenario"] == "Clear/Clear-VB0"]

#baseline_annual

In [18]:
annualEnergy_fan = baseline_annual["FanEnergy(kBtu)"].sum()
annualEnergy_fan

2518.5942054690404

In [19]:
annualEnergy_cool = baseline_annual["CoolingEnergy(kBtu)"].sum()
annualEnergy_cool

9148.732496988963

In [20]:
annualEnergy_heat = baseline_annual["HeatingEnergy(kBtu)"].sum()
annualEnergy_heat

106490.68918688194

In [21]:
annualEnergy = (annualEnergy_fan + annualEnergy_cool + annualEnergy_heat)
annualEnergy

118158.01588933993

In [22]:
annualCost_fan = baseline_annual["FanCost($)"].sum()
annualCost_fan

96.98990093485853

In [23]:
annualCost_cool = baseline_annual["CoolingCost($)"].sum()
annualCost_cool

352.31346782092317

In [24]:
annualCost_heat = baseline_annual["HeatingCost($)"].sum()
annualCost_heat

883.87272025112

In [25]:
annualCost = (annualCost_fan + annualCost_cool + annualCost_heat)
annualCost

1333.1760890069017

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Add baselines to dataframe<br>
Export to file

In [26]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario"], as_index=True)

#annualData.sum().round(2)

In [27]:
annualDataFinal = annualData.sum().round(2)

#annualDataFinal

In [28]:
# Add columns for delta from base
annualDataFinal["DeltaEnergyFromBase"] = annualDataFinal["AnnualEnergy(kBtu)"] - annualEnergy
annualDataFinal["DeltaCostFromBase"] = annualDataFinal["AnnualCost($)"] - annualCost

#annualDataFinal.round(2)

In [29]:
# Export grouped file to csv
annualDataFinal.to_csv("../Scraper_Output/msp_annual_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh<br>
Drop unwanted columns

In [30]:
# Copy dataframe to modify and leave original df intact
get_monthly = all_data2.copy(deep=True)

#get_monthly

In [31]:
# Convert Joules to kWh and therms in new dataframe
get_monthly["FanEnergy(kWh)"] = get_monthly["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly["HeatingEnergy(therms)"] = get_monthly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly["CoolingEnergy(kWh)"] = get_monthly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly

In [32]:
# Drop columns
get_monthly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [33]:
# Copy dataframe to modify and leave original df intact
get_monthly_costs = all_data2.copy(deep=True)

#get_monthly

In [34]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly_costs["FanCost($)"] = (get_monthly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly_costs["HeatingCost($)"] = ((get_monthly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly_costs["CoolingCost($)"] = (get_monthly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [35]:
# Drop columns
get_monthly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Establish baseline <br>
Export combine file

In [36]:
# Combine data for energy and cost
allMonthlyData = pd.merge(get_monthly, get_monthly_costs, how="inner" )

#allMonthlyData

In [37]:
# Add cost totals, don't total energy...in different units
totalCostMonthly = allMonthlyData["FanCost($)"] + allMonthlyData["HeatingCost($)"] + allMonthlyData["CoolingCost($)"]
allMonthlyData["MonthlyCost($)"] = totalCostMonthly

#allMonthlyData

In [38]:
allMonthlyData["Month"] = allMonthlyData.Date_Time.str.slice(0,3)

#allMonthlyData


### Establish Monthly Energy and Cost Baseline Data


In [39]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_monthly = allMonthlyData[allMonthlyData["Scenario"] == "Clear/Clear-VB0"]

baseline_monthly

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanCost($),HeatingCost($),CoolingCost($),MonthlyCost($),Month
385440,Clear/Clear-VB0,01/01 01:00:00,0.0,0.129081,0.284288,0.0,0.016961,0.235959,0.0,0.252920,01
385441,Clear/Clear-VB0,01/01 02:00:00,0.0,0.144110,0.317387,0.0,0.018936,0.263432,0.0,0.282368,01
385442,Clear/Clear-VB0,01/01 03:00:00,0.0,0.147442,0.324726,0.0,0.019374,0.269523,0.0,0.288897,01
385443,Clear/Clear-VB0,01/01 04:00:00,0.0,0.146993,0.323736,0.0,0.019315,0.268701,0.0,0.288016,01
385444,Clear/Clear-VB0,01/01 05:00:00,0.0,0.149455,0.329159,0.0,0.019638,0.273202,0.0,0.292841,01
...,...,...,...,...,...,...,...,...,...,...,...
394195,Clear/Clear-VB0,12/31 20:00:00,0.0,0.088982,0.195974,0.0,0.011692,0.162658,0.0,0.174350,12
394196,Clear/Clear-VB0,12/31 21:00:00,0.0,0.088945,0.195893,0.0,0.011687,0.162591,0.0,0.174278,12
394197,Clear/Clear-VB0,12/31 22:00:00,0.0,0.092107,0.202857,0.0,0.012103,0.168371,0.0,0.180474,12
394198,Clear/Clear-VB0,12/31 23:00:00,0.0,0.096148,0.211756,0.0,0.012634,0.175758,0.0,0.188392,12


In [40]:
baseline_monthly["Month"] = baseline_monthly.Date_Time.str.slice(0,3)

baseline_monthly

C:\Users\khagl\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanCost($),HeatingCost($),CoolingCost($),MonthlyCost($),Month
385440,Clear/Clear-VB0,01/01 01:00:00,0.0,0.129081,0.284288,0.0,0.016961,0.235959,0.0,0.252920,01
385441,Clear/Clear-VB0,01/01 02:00:00,0.0,0.144110,0.317387,0.0,0.018936,0.263432,0.0,0.282368,01
385442,Clear/Clear-VB0,01/01 03:00:00,0.0,0.147442,0.324726,0.0,0.019374,0.269523,0.0,0.288897,01
385443,Clear/Clear-VB0,01/01 04:00:00,0.0,0.146993,0.323736,0.0,0.019315,0.268701,0.0,0.288016,01
385444,Clear/Clear-VB0,01/01 05:00:00,0.0,0.149455,0.329159,0.0,0.019638,0.273202,0.0,0.292841,01
...,...,...,...,...,...,...,...,...,...,...,...
394195,Clear/Clear-VB0,12/31 20:00:00,0.0,0.088982,0.195974,0.0,0.011692,0.162658,0.0,0.174350,12
394196,Clear/Clear-VB0,12/31 21:00:00,0.0,0.088945,0.195893,0.0,0.011687,0.162591,0.0,0.174278,12
394197,Clear/Clear-VB0,12/31 22:00:00,0.0,0.092107,0.202857,0.0,0.012103,0.168371,0.0,0.180474,12
394198,Clear/Clear-VB0,12/31 23:00:00,0.0,0.096148,0.211756,0.0,0.012634,0.175758,0.0,0.188392,12


In [41]:
# Group by scenario and date/time for export to graphing
baseline_monthly_all = baseline_monthly.groupby(["Month"], as_index=False).sum()

baseline_monthly_all

,Month,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanCost($),HeatingCost($),CoolingCost($),MonthlyCost($)
0,01,863062.838677,123.481090,271.954336,0.000000,16.225415,225.722099,0.000000,241.947514
1,02,439661.106454,76.975698,169.531017,0.000000,10.114607,140.710744,0.000000,150.825351
2,03,458072.653237,53.448594,116.807008,3.851997,7.023145,96.949817,0.506152,104.479115
3,04,441383.426370,37.543794,64.194870,87.610774,4.933255,53.281742,11.512056,69.727052
4,05,458332.737111,30.960764,9.873026,272.913961,4.068244,8.194612,35.860894,48.123751
5,06,452845.181610,52.698153,1.639673,540.620049,6.924537,1.360928,71.037474,79.322940
6,07,455399.808794,81.916086,0.000000,885.880535,10.763774,0.000000,116.404702,127.168476
7,08,473569.855359,55.264156,0.282724,578.672427,7.261710,0.234661,76.037557,83.533928
8,09,432456.317898,34.903700,14.470172,293.152749,4.586346,12.010242,38.520271,55.116860
9,10,415077.653640,25.896133,52.715066,18.353773,3.402752,43.753505,2.411686,49.567943


In [42]:
# Rename Columns
baseline_monthly_all = baseline_monthly_all.rename(columns={"Month":"Month",
                              "FanEnergy(kWh)":"Base_FanEnergy(kWh)",
                             "HeatingEnergy(therms)":"Base_HeatingEnergy(therms)",
                             "CoolingEnergy(kWh)":"Base_CoolingEnergy(kWh)",
                             "FanCost($)":"Base_FanCost($)",
                             "HeatingCost($)":"Base_HeatingCost($)",
                             "CoolingCost($)":"CoolingCost($)",
                             "MonthlyCost($)":"Base_MonthlyCost($)"
                             })
#baseline_monthly_all


In [43]:
# Drop Illuminance column...don't need the totals
baseline_monthly_all.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

baseline_monthly_all

,Month,Base_FanEnergy(kWh),Base_HeatingEnergy(therms),Base_CoolingEnergy(kWh),Base_FanCost($),Base_HeatingCost($),CoolingCost($),Base_MonthlyCost($)
0,01,123.481090,271.954336,0.000000,16.225415,225.722099,0.000000,241.947514
1,02,76.975698,169.531017,0.000000,10.114607,140.710744,0.000000,150.825351
2,03,53.448594,116.807008,3.851997,7.023145,96.949817,0.506152,104.479115
3,04,37.543794,64.194870,87.610774,4.933255,53.281742,11.512056,69.727052
4,05,30.960764,9.873026,272.913961,4.068244,8.194612,35.860894,48.123751
5,06,52.698153,1.639673,540.620049,6.924537,1.360928,71.037474,79.322940
6,07,81.916086,0.000000,885.880535,10.763774,0.000000,116.404702,127.168476
7,08,55.264156,0.282724,578.672427,7.261710,0.234661,76.037557,83.533928
8,09,34.903700,14.470172,293.152749,4.586346,12.010242,38.520271,55.116860
9,10,25.896133,52.715066,18.353773,3.402752,43.753505,2.411686,49.567943


In [44]:
# Export grouped file to csv to be used as the baseline data
baseline_monthly_all.round(2).to_csv("../Scraper_Output/msp_monthly_baseline.csv", header=True, index=False)

## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Add baselines to dataframe<br>
Export to file

In [45]:
# Group by scenario and date/time for export to graphing
MonthlyData2 = allMonthlyData.groupby(["Scenario", "Month"], as_index=True)

#MonthlyData.sum()

In [46]:
MonthlyDataFinal = MonthlyData2.sum()

#MonthlyDataFinal

In [47]:
# Drop Illuminance column...don't need the totals
MonthlyDataFinal.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

In [48]:
# Export grouped file to csv (used before grouped by month)
MonthlyDataFinal.round(2).to_csv("../Scraper_Output/msp_monthly_data.csv", header=True, index=True)

## Get Monthly Average Illuminance
Get just the illuminance column<br>
Group by scenario <br>
Groups by month <br>

In [49]:
Illuminance = allMonthlyData[["Scenario", "Date_Time", "Illuminance(lux)", "Month"]]

#Illuminance

In [50]:
# Group by scenario
MonthlyIlluminance = Illuminance.groupby(["Scenario", "Month"])

#MonthlyIlluminance.mean()

In [51]:
# Export grouped file to csv
MonthlyIlluminance.mean().round(2).to_csv("../Scraper_Output/msp_monthly_illuminance.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to therms and kWh<br>
Drop unwanted columns

In [52]:
# Copy dataframe to modify and leave original df intact
all_data3 = all_data1.copy(deep=True)

#all_data3

In [53]:
# Get wanted columns
# Include hourly illumance
all_data3 = all_data3[["Scenario",
               "Date/Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "
                      ]]

#all_data3

In [54]:
# Rename Columns
get_hourly = all_data3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly

In [55]:
# Convert Joules to kWh and therms in new dataframe
get_hourly["FanEnergy(kWh)"] = get_hourly["FanEnergy[J](Hourly)"] * .000000277777778
get_hourly["HeatingEnergy(therms)"] = get_hourly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_hourly["CoolingEnergy(kWh)"] = get_hourly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_hourly

In [56]:
# Drop columns
get_hourly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly

------

## Get Hourly Cost Data
Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [57]:
# Copy dataframe to modify and leave original df intact
get_hourly_costs = all_data3.copy(deep=True)

#get_hourly_costs

In [58]:
# Rename Columns
get_hourly_costs = get_hourly_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly_costs

In [59]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly_costs["FanEnergy($)"] = (get_hourly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly_costs["HeatingEnergy($)"] = ((get_hourly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly_costs["CoolingEnergy($)"] = (get_hourly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [60]:
# Drop columns
get_hourly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Establish baseline <br>
Export combine file

In [61]:
# Combine data for energy and cost
allHourlyData = pd.merge(get_hourly, get_hourly_costs, how="inner")

#allHourlyData

In [62]:
# Add energy cost totals

totalCosthourly = allHourlyData["FanEnergy($)"] + allHourlyData["HeatingEnergy($)"] + allHourlyData["CoolingEnergy($)"]
allHourlyData["HourlyCost($)"] = totalCosthourly

#allHourlyData


### Establish Hourly Energy and Cost Baseline Data


In [63]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_hourly = allHourlyData[allHourlyData["Scenario"] == "Clear/Clear-VB0"]

#baseline_hourly

In [64]:
hourlyEnergy_fan = baseline_hourly["FanEnergy(kWh)"].mean()
hourlyEnergy_fan

0.08426108447041908

In [65]:
hourlyEnergy_cool = baseline_hourly["CoolingEnergy(kWh)"].mean()
hourlyEnergy_cool

0.3060763500734302

In [66]:
hourlyEnergy_heat = baseline_hourly["HeatingEnergy(therms)"].mean()
hourlyEnergy_heat

0.12156471368365519

In [67]:
hourlyCost_fan = baseline_hourly["FanEnergy($)"].mean()
hourlyCost_fan

0.011071906499413016

In [68]:
hourlyCost_cool = baseline_hourly["CoolingEnergy($)"].mean()
hourlyCost_cool

0.04021843239964872

In [69]:
hourlyCost_heat = baseline_hourly["HeatingEnergy($)"].mean()
hourlyCost_heat

0.10089871235743367

In [70]:
hourlyCost = (hourlyCost_fan + hourlyCost_cool + hourlyCost_heat)
hourlyCost

0.1521890512564954

## Export Hourly Data

Add baselines to dataframe<br>
Export to file

In [71]:
# Add columns for delta from base
allHourlyData["DeltaFanFromBase"] = allHourlyData["FanEnergy(kWh)"] - hourlyEnergy_fan
allHourlyData["DeltaCoolFromBase"] = allHourlyData["CoolingEnergy(kWh)"] - hourlyEnergy_cool
allHourlyData["DeltaHeatFromBase"] = allHourlyData["HeatingEnergy(therms)"] - hourlyEnergy_heat
allHourlyData["DeltaCostFromBase"] = allHourlyData["HourlyCost($)"] - hourlyCost

allHourlyData.round(2)

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),HourlyCost($),DeltaFanFromBase,DeltaCoolFromBase,DeltaHeatFromBase,DeltaCostFromBase
0,Clear/lowE(3)-VB-15,01/01 01:00:00,0.0,0.12,0.26,0.0,0.02,0.22,0.0,0.23,0.04,-0.31,0.14,0.08
1,Clear/lowE(3)-VB-15,01/01 02:00:00,0.0,0.13,0.30,0.0,0.02,0.25,0.0,0.26,0.05,-0.31,0.17,0.11
2,Clear/lowE(3)-VB-15,01/01 03:00:00,0.0,0.14,0.30,0.0,0.02,0.25,0.0,0.27,0.05,-0.31,0.18,0.12
3,Clear/lowE(3)-VB-15,01/01 04:00:00,0.0,0.14,0.30,0.0,0.02,0.25,0.0,0.27,0.05,-0.31,0.18,0.12
4,Clear/lowE(3)-VB-15,01/01 05:00:00,0.0,0.14,0.31,0.0,0.02,0.26,0.0,0.27,0.06,-0.31,0.19,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716955,lowE(2)/lowE(4)-BBG75,12/31 20:00:00,0.0,0.08,0.18,0.0,0.01,0.15,0.0,0.16,-0.00,-0.31,0.06,0.01
1716956,lowE(2)/lowE(4)-BBG75,12/31 21:00:00,0.0,0.08,0.18,0.0,0.01,0.15,0.0,0.16,-0.00,-0.31,0.06,0.01
1716957,lowE(2)/lowE(4)-BBG75,12/31 22:00:00,0.0,0.08,0.18,0.0,0.01,0.15,0.0,0.16,-0.00,-0.31,0.06,0.01
1716958,lowE(2)/lowE(4)-BBG75,12/31 23:00:00,0.0,0.09,0.19,0.0,0.01,0.16,0.0,0.17,0.00,-0.31,0.07,0.02


In [72]:
# Export grouped file to csv for daily cost sums
allHourlyData.to_csv("../Scraper_Output/msp_daily.csv", header=True, index=False)